In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.cluster import KMeans, AgglomerativeClustering
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import time
import h5py

In [88]:
df = pd.read_csv("/Users/jpate/Downloads/netflix_titles.csv")

In [89]:
df = df.dropna(axis = 0)

In [90]:
df = df[df["release_year"]>=2000]

In [91]:
df = df[(df["rating"] != "NC-17") & (df["rating"] != "UR") & (df["rating"] != "TV-Y7-FV")]

In [93]:
df = df.reset_index(drop=True)

In [94]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
1,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
2,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
3,s28,Movie,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,"September 20, 2021",2010,PG-13,103 min,Comedies,Mourning the loss of their beloved junior high...
4,s29,Movie,Dark Skies,Scott Stewart,"Keri Russell, Josh Hamilton, J.K. Simmons, Dak...",United States,"September 19, 2021",2013,PG-13,97 min,"Horror Movies, Sci-Fi & Fantasy",A family’s idyllic suburban life shatters when...
...,...,...,...,...,...,...,...,...,...,...,...,...
4879,s8802,Movie,Zinzana,Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...","United Arab Emirates, Jordan","March 9, 2016",2015,TV-MA,96 min,"Dramas, International Movies, Thrillers",Recovering alcoholic Talal wakes up inside a s...
4880,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
4881,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
4882,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [96]:
df_titles = df.copy()

In [99]:
df_titles = df_titles.drop(['show_id','director','cast','country','date_added', 'release_year','type','rating','duration','listed_in','description'], axis=1)

In [172]:
df_titles.head(1200)

,title
0,The Great British Baking Show
1,The Starling
2,Je Suis Karl
3,Grown Ups
4,Dark Skies
...,...
1195,Chippa
1196,Cook Off
1197,Kalek Shanab
1198,Mirai


In [113]:
titles = df_titles.to_dict()

In [122]:
titles = titles['title']

In [123]:
titles[0]

'The Great British Baking Show'

In [31]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.3.0/en_core_web_lg-3.3.0.tar.gz

     |████████████████████████████████| 400.6 MB 5.0 kB/s eta 0:00:012


In [7]:
nlp = spacy.load('en_core_web_lg')

In [11]:
doc = nlp("the dog ran slowly")

In [19]:
doc2 = nlp("the cat laid down")

In [20]:
doc.similarity(doc2)

0.8646943903180406

In [124]:
similarities = {}

In [174]:
similarities

{0: {0: 1.0,
  1: 0.9022030061187029,
  2: 0.8802959748246284,
  3: 0.9009243168360356,
  4: 0.861987168439197,
  5: 0.8485952626145536,
  6: 0.8244822848926722,
  7: 0.8625231408224138,
  8: 0.8564161096157192,
  9: 0.8785862339049453,
  10: 0.8812917706132765,
  11: 0.8418526569807553,
  12: 0.8947390063255998,
  13: 0.8184350987688677,
  14: 0.8548332587186345,
  15: 0.8950871899425142,
  16: 0.8565774279676966,
  17: 0.87839238181012,
  18: 0.8557672301224047,
  19: 0.7399447004777944,
  20: 0.8741382143102632,
  21: 0.8482221195570715,
  22: 0.902947545958047,
  23: 0.8801683369350137,
  24: 0.8977839314031376,
  25: 0.8577472076929836,
  26: 0.8242922244306654,
  27: 0.9114896827569975,
  28: 0.8700537890701397,
  29: 0.8340227370483566,
  30: 0.8697743812494501,
  31: 0.9041412937587835,
  32: 0.8522300293708522,
  33: 0.8728454391979726,
  34: 0.8885746734265181,
  35: 0.8473640881218607,
  36: 0.8420429093609816,
  37: 0.8585163972991089,
  38: 0.8473279223353067,
  39: 0.8939

In [164]:
def func():
    for i, row in df.iterrows():
        doc = nlp(row['description'])
        similarities[i] = {}
    #     print(row['title'])
        for j, row2 in df.iterrows():
            doc2 = nlp(row2['description'])
            #print(f"{row['title']} x {row2['title']}: {doc.similarity(doc2):.10f}")
            similarities[i][j] = doc.similarity(doc2)

In [ ]:
func()

In [131]:
similarities[0][1]

0.9022030061187029

In [136]:
def lookup_by_title(title):
    for title_id in titles:
        if titles[title_id] == title:
            return title_id
    return None

In [144]:
def lookup(title1, title2):
    return similarities[lookup_by_title(title1)][lookup_by_title(title2)]

In [173]:
#lookup('Mirai', "Zoom")

KeyError: 1198